# Imports

In [1]:
from ultralytics import YOLO
import os

In [9]:
# load model
model = YOLO("yolov8m-pose.pt")

# configurations
N = 10
MPII_IMAGES_DIR = "./mpii/images"
IMAGE_FILES = [os.path.join(MPII_IMAGES_DIR, file) for file in os.listdir(MPII_IMAGES_DIR) if file.endswith(('.jpg'))][:N]

# iterate for each image
for image_file in IMAGE_FILES:
    # execute inference
    results = model(source= image_file, show = True, conf = 0.3, save = True)

    del results.xyxy

    del model


image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000001163.jpg: 384x640 2 persons, 617.5ms
Speed: 0.0ms preprocess, 617.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\pose\predict


AttributeError: 'list' object has no attribute 'xyxy'